# State Probability All Probabilities FCFS Example

This example demonstrates how to compute all state probabilities in a FCFS queueing network.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create network with FCFS queues
val model = Network("myModel")

// Block 1: nodes (FCFS scheduling)
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.FCFS)
val node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
node2.setNumberOfServers(1)
node3.setNumberOfServers(1)

In [ ]:
// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 3, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)

In [ ]:
// Set service times
node1.setService(jobclass1, Exp.fitMean(1.0))
node1.setService(jobclass2, Exp.fitMean(1.0))

node2.setService(jobclass1, Exp.fitMean(0.5))
node2.setService(jobclass2, Exp.fitMean(0.8))

node3.setService(jobclass1, Exp.fitMean(1.2))
node3.setService(jobclass2, Exp.fitMean(0.7))

In [ ]:
// Block 3: routing
val P = model.initRoutingMatrix()

// Cyclic routing
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)

P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)

model.link(P)

In [ ]:
// Solve with CTMC to get all state probabilities
println("Solving with CTMC for all state probabilities:")
try {
    val solverCtmc = CTMC(model)
    val avgTable = solverCtmc.avgTable
    println("CTMC Results:")
    avgTable.print()
    
    // Get all state probabilities
    println("\nTesting all state probabilities methods:")
    try {
        val allProbs = solverCtmc.probSysAggr
        println("System aggregated probabilities computed successfully")
        println("Number of states: ${allProbs.size}")
    } catch (e: Exception) {
        println("All probabilities method not available: ${e.message}")
    }
    
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

In [ ]:
// Test with NC solver for comparison
println("\nTesting with NC solver:")
try {
    val solverNc = NC(model)
    val avgTableNc = solverNc.avgTable
    println("NC Results:")
    avgTableNc.print()
} catch (e: Exception) {
    println("NC solver error: ${e.message}")
}